In [9]:
import skimage.io as io
import numpy as np
from skimage.filters import threshold_otsu 
from skimage.color import rgb2grey
from skimage.measure import find_contours
import cv2
from windows import getWindows

In [10]:
def First_difference(chain_code):
    #chain =chain_code[-1] +chain_code
    chain = chain_code
    chain = list(chain)
    chain_shifted = chain[1:]
    chain = chain[:-1]
    chain = np.array(chain).astype(np.int)
    chain_shifted = np.array(chain_shifted).astype(np.int)
    first_diff = -1*(chain - chain_shifted)%8
    first_diff = first_diff.astype(np.str)
    return "".join(first_diff)
    

In [11]:
def Chain_Codes_Counting(contours,pairs_triplets=True):
    global_chaining_code = ""
    first_difference = ""
    second_difference = ""
    if(pairs_triplets):
        F4 = np.zeros(64)
        F5 = np.zeros(512)
    
    for j in range(len(contours)):
        c = contours[j]
        local_chaining_code = ""
        for i in range(1,len(c)):
            diff = (c[i]-c[i-1])[0]
            if(diff[0]==1 and diff[1]==0):
                local_chaining_code +="0"
            elif(diff[0]==1 and diff[1]==-1):
                local_chaining_code+="1"
            elif(diff[0]==0 and diff[1]==-1):
                local_chaining_code +="2"
            elif(diff[0]==-1 and diff[1]==-1):
                local_chaining_code+="3"
            elif(diff[0]==-1 and diff[1]==0):
                local_chaining_code+="4"
            elif(diff[0]==-1 and diff[1]==1):
                local_chaining_code +="5"
            elif(diff[0]==0 and diff[1]==1):
                local_chaining_code+="6"
            elif(diff[0]==1 and diff[1]==1):
                local_chaining_code+="7"
            else:
                print("ERROR")

        if(len(local_chaining_code)>0):
            first_diff = First_difference(local_chaining_code)
            first_difference += first_diff    
            second_difference += First_difference(first_diff)
        global_chaining_code+=local_chaining_code
        for  i in range(8):
            for j in range(8):
                check = str(i)+str(j)
                F4[i*8+j] += local_chaining_code.count(check)
                for k in range(8):
                    check = str(i)+str(j)+str(k)
                    F5[i*64+j*8+k] += local_chaining_code.count(check)
    F1  = []
    F2  = []
    F3  = []
    for i in range(8):
        F1.append(global_chaining_code.count(str(i)))
        if i!=5:
            F2.append(first_difference.count(str(i)))

        F3.append(second_difference.count(str(i)))

    F1 = np.array(F1)
    F2 = np.array(F2)
    F3 = np.array(F3)
    F1 = F1/F1.sum()
    F2 = F2/F2.sum()
    F3 = F3/F3.sum()
    if pairs_triplets:
        F4 = F4/F4.sum()
        F5 = F5/F5.sum()
        return F1,F2,F3,F4,F5
    return F1,F2,F3

In [14]:
def Chain_Codes_Feature_Extractor(image_name):
    img = io.imread(image_name)
    windows = getWindows(img,13)
    print(windows.shape)
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    
    #img = np.ones(img.shape)*255
    #cv2.drawContours(img, contours, -1, (30, 255, 30), 1) 
    
    F1,F2,F3,F4,F5 = Chain_Codes_Counting(contours,True)
    
    
    
    return F1,F2,F3,F4,F5
    
    




   

    
    

In [15]:
F1,F2,F3,F4,F5 = Chain_Codes_Feature_Extractor('TestCases/0.png')

(4844, 13, 13)
